In [1]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
import sqlite3
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from src import paths,sql_querys,converting, pipelines, square_sum, split
from sklearn.metrics import mean_absolute_error
import joblib


In [2]:
conn = sqlite3.connect(paths.db_path)
cursor = conn.cursor()
query = sql_querys.query_age_model
df = pd.read_sql_query(query, conn)
df.max_balance = pd.to_numeric(df.max_balance)
df = converting.converter(df)


,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,max_balance,currency
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,2775.615374,CZK
1,1222646323,46.0,services,divorced,unknown,M,no,no,n,no,1054.517907,CZK
2,451375919,33.0,admin.,single,secondary,F,no,no,n,yes,1303.494818,CZK
3,338972671,44.0,self-employed,married,secondary,F,no,no,y,yes,446.676191,CZK
4,1472834688,36.0,blue-collar,married,primary,M,yes,no,n,yes,2011.939205,CZK


In [3]:
df1 = df.drop(columns='client_id')
df1.dropna(subset=['age'], inplace=True, how='any', axis=0)
target_column = 'age'
X_train, X_test, y_train, y_test = split.split(df1,target_column)

In [4]:
GBR_params = {'learning_rate':0.05, 'loss':'huber', 'max_depth':8,  'max_features':'sqrt', 'min_samples_leaf':10, 'min_samples_split':10, 'n_estimators':129}
num_features = ['max_balance']
cat_features = ['job']
labeled_features = ['marital', 'education', 'gender', 'has_deposits', 'loan', 'has_insurance', 'has_mortgage']

pipeline = pipelines.age_pipeline(num_features=num_features,cat_features=cat_features,labeled_features=labeled_features,GBR_params=GBR_params)

In [5]:
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

# Print the MAE
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 6.576571898855847


In [7]:
joblib.dump(pipeline, paths.age_pipeline_path)

['c:\\Users\\spiri\\codingbootcamp\\Final-project-C4Y\\pipelines\\age_pipeline.pkl']